<a href="https://colab.research.google.com/github/sbanerjee2020/Traderbot/blob/master/Tensorboard_trial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### https://www.dlology.com/blog/quick-guide-to-run-tensorboard-in-google-colab/

### In Notebook settings select GPU

In [1]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

--2019-04-01 03:57:17--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.73.9.93, 52.22.145.207, 52.21.103.149, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.73.9.93|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14977695 (14M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  14.28M  67.1MB/s    in 0.2s    

2019-04-01 03:57:22 (67.1 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [14977695/14977695]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [0]:
LOG_DIR = './log'
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)

In [0]:
get_ipython().system_raw('./ngrok http 6006 &')

In [4]:
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

https://4f0f160d.ngrok.io


### Click on the link above when the ConvNet is running and you can see the progress on Tensorboard

In [5]:
'''Trains a simple convnet on the MNIST dataset.

Gets to 99.25% test accuracy after 12 epochs
(there is still a lot of margin for parameter tuning).
16 seconds per epoch on a GRID K520 GPU.
'''

from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from keras.callbacks import TensorBoard

batch_size = 128
num_classes = 10
epochs = 12

# input image dimensions
img_rows, img_cols = 28, 28

# the data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])


tbCallBack = TensorBoard(log_dir='./log', histogram_freq=1,
                         write_graph=True,
                         write_grads=True,
                         batch_size=batch_size,
                         write_images=True)

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test),
          callbacks=[tbCallBack])
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])


Using TensorFlow backend.


11493376/11490434 [==============================] - 0s 0us/step
x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Train on 60000 samples, validate on 10000 samples
Epoch 1/12
60000/60000 [==============================] - 12s 201us/step - loss: 0.2478 - acc: 0.9230 - val_loss: 0.0530 - val_acc: 0.9828
Epoch 2/12
60000/60000 [==============================] - 9s 146us/step - loss: 0.0829 - acc: 0.9748 - val_loss: 0.0405 - val_acc: 0.9862
Epoch 3/12
60000/60000 [==============================] - 9s 146us/step - loss: 0.0609 - acc: 0.9813 - val_loss: 0.0345 - val_acc: 0.9880
Epoch 4/12
60000/60000 [==============================] - 9s 147us/step - loss: 0.0512 - acc: 0.9844 - val_loss: 0.0350 - val_acc: 0.9882
Epoch 5/12
60000/60000 [

In [6]:
!ps aux | grep tensorboard

root         160  2.9  2.5 1808356 346000 ?      Sl   03:57   0:14 /usr/bin/python2 /usr/local/bin/tensorboard --logdir ./log --host 0.0.0.0 --port 6006
root         281  0.0  0.0  39196  6588 ?        S    04:05   0:00 /bin/bash -c ps aux | grep tensorboard
root         283  0.0  0.0  16128   624 ?        R    04:05   0:00 grep tensorboard


In [7]:
!ps aux | grep ngrok

root         168  0.3  0.3 242048 46768 ?        Sl   03:57   0:01 ./ngrok http 6006
root         286  0.0  0.0  39196  6576 ?        S    04:05   0:00 /bin/bash -c ps aux | grep ngrok
root         288  0.0  0.0  38568  5008 ?        S    04:05   0:00 grep ngrok


In [0]:
# Can kill a process with !kill PID
# Clear tensorboard log with !rm -rf log